In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from sklearn.metrics import confusion_matrix
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from imutils import paths
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

Using TensorFlow backend.


In [2]:
trainPath = '../PFR_Data/train/'
testPath = '../PFR_Data/test/'
valPath = '../PFR_Data/vald'
BATCH_SIZE = 128
PFR_NUM_CLASS = 10
FUEL_TYPE_NUM_CLASS = 5
EPOCHS = 150

In [3]:
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

In [4]:
def plot_training(H, N, plotPath):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

## Augmentation

In [5]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator('''
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"''')
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

## Generator

In [6]:
# initialize the training generator
img_size = 224
trainGen = trainAug.flow_from_directory(
    trainPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)
# initialize the validation generator
valGen = valAug.flow_from_directory(
    valPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)
# initialize the testing generator
testGen = valAug.flow_from_directory(
    testPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)

Found 8555 images belonging to 10 classes.
Found 1053 images belonging to 10 classes.
Found 1053 images belonging to 10 classes.


In [7]:
# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(img_size, img_size, 3)))
# construct the head of the model that will be placed on top of the the base model


In [8]:
#baseModel =  pickle.load(open('../Models/VGG16.sav', 'rb'))

In [9]:
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(PFR_NUM_CLASS, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [10]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [11]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(trainGen.classes), 
                trainGen.classes)

In [12]:
class_weights

array([0.18517316, 2.06642512, 1.00410798, 3.52057613, 2.4869186 ,
       2.27526596, 1.57261029, 6.68359375, 1.04202192, 4.01643192])

In [ ]:
# compile our model (this needs to be done after our setting our layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
#initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weights)
# reset the testing generator and evaluate the network after
# fine-tuning just the network head

[INFO] compiling model...
[INFO] training head...
Epoch 1/150
66/66 [==============================] - 109s 2s/step - loss: 7.5586 - accuracy: 0.4907 - val_loss: 1.1135 - val_accuracy: 0.6963
Epoch 2/150
66/66 [==============================] - 86s 1s/step - loss: 1.1907 - accuracy: 0.6077 - val_loss: 2.0509 - val_accuracy: 0.7589
Epoch 3/150
66/66 [==============================] - 87s 1s/step - loss: 0.9895 - accuracy: 0.6477 - val_loss: 1.7461 - val_accuracy: 0.7697
Epoch 4/150
66/66 [==============================] - 90s 1s/step - loss: 0.8446 - accuracy: 0.6931 - val_loss: 0.6759 - val_accuracy: 0.7870
Epoch 5/150
66/66 [==============================] - 94s 1s/step - loss: 0.7427 - accuracy: 0.7395 - val_loss: 0.0276 - val_accuracy: 0.7481
Epoch 6/150
66/66 [==============================] - 96s 1s/step - loss: 0.7869 - accuracy: 0.7039 - val_loss: 0.0536 - val_accuracy: 0.7416
Epoch 7/150
66/66 [==============================] - 86s 1s/step - loss: 0.5979 - accuracy: 0.7754 - va

66/66 [==============================] - 88s 1s/step - loss: 0.1480 - accuracy: 0.9426 - val_loss: 0.4588 - val_accuracy: 0.9243
Epoch 59/150
66/66 [==============================] - 77s 1s/step - loss: 0.1459 - accuracy: 0.9424 - val_loss: 0.0224 - val_accuracy: 0.8984
Epoch 60/150
66/66 [==============================] - 82s 1s/step - loss: 0.1526 - accuracy: 0.9384 - val_loss: 1.2353e-04 - val_accuracy: 0.8714
Epoch 61/150
66/66 [==============================] - 85s 1s/step - loss: 0.1603 - accuracy: 0.9346 - val_loss: 1.6364e-05 - val_accuracy: 0.8886
Epoch 62/150
66/66 [==============================] - 80s 1s/step - loss: 0.1408 - accuracy: 0.9455 - val_loss: 0.0019 - val_accuracy: 0.8908
Epoch 63/150
66/66 [==============================] - 82s 1s/step - loss: 0.1449 - accuracy: 0.9429 - val_loss: 0.0339 - val_accuracy: 0.8854
Epoch 64/150
66/66 [==============================] - 87s 1s/step - loss: 0.1358 - accuracy: 0.9496 - val_loss: 0.4239 - val_accuracy: 0.9053
Epoch 65/15

In [38]:
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
    steps=(totalTest // BATCH_SIZE) + 1)
predIdxsClasses = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxsClasses,
    target_names=testGen.class_indices.keys()))

[INFO] evaluating after fine-tuning network head...
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       576
         1-3       1.00      0.38      0.55        50
       10-12       0.90      0.94      0.92       105
       13-15       0.83      0.66      0.73        29
       20-25       0.81      0.95      0.88        41
       30-40       0.88      0.98      0.93        46
         4-5       0.62      0.92      0.74        66
       41-56       1.00      0.93      0.96        14
         6-9       1.00      0.74      0.85       101
         60+       0.96      0.92      0.94        25

    accuracy                           0.92      1053
   macro avg       0.90      0.84      0.85      1053
weighted avg       0.93      0.92      0.92      1053



In [18]:
labs = testGen.class_indices.keys()
PFRLabels = []
for lab in labs:
    PFRLabels.append(lab)
PFRLabels

['0', '1-3', '10-12', '13-15', '20-25', '30-40', '4-5', '41-56', '6-9', '60+']

In [19]:
cm = confusion_matrix(testGen.classes, predIdxsClasses)
cm = pd.DataFrame(cm)
cm.columns = PFRLabels
cm.index = PFRLabels
cm

,0,1-3,10-12,13-15,20-25,30-40,4-5,41-56,6-9,60+
0,575,0,0,0,0,0,1,0,0,0
1-3,10,32,0,0,1,0,7,0,0,0
10-12,0,0,94,3,0,0,0,0,8,0
13-15,1,0,3,22,0,3,0,0,0,0
20-25,1,0,0,3,33,4,0,0,0,0
30-40,0,0,0,0,0,43,0,0,0,3
4-5,5,2,0,0,0,0,58,0,1,0
41-56,0,0,0,0,0,3,0,11,0,0
6-9,2,2,3,0,0,0,29,0,64,1
60+,0,0,0,0,0,1,0,0,0,24


In [21]:
WARMUP_PLOT_PATH = '../Models/PFRModel/train.png'
plot_training(H, EPOCHS, WARMUP_PLOT_PATH)

In [22]:
pickle.dump(model, open('../Models/PFRModel/PFRModel.sav', 'wb'))